<a href="https://colab.research.google.com/github/wwidjaja0/ucla-aji-7/blob/main/UCLA_AJL_Team_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **UCLA AJI Team 7**



---


**Resources and Documents:**  
[Notion](https://www.notion.so/team/18610071-e06a-812f-98e5-004281f76263/join) | [Google Drive](https://drive.google.com/drive/folders/1uNRJqQwA9hGX1WO9Tq-dfMP-ASt_dR5L?usp=sharing) | [Github Repo](https://github.com/wwidjaja0/ucla-aji-7)

---

**Summary**

## **How to contribute to this Colab**


First, we shall import the datasets. This includes:  
1. X

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
# import warnings
# warnings.filterwarnings("ignore")

Load the CSV files